In [74]:
import psycopg2
import pandas as pd
import numpy as np
import mariadb
import json
import os
import shutil
import subprocess
from pathlib import Path
import pyodbc
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from scipy.spatial import distance
import pickle
import matplotlib.pyplot as plt
from sklearn.impute import SimpleImputer
import seaborn as sns

In [75]:
water_df = pd.read_csv("output/water_analysis_cleaned.csv")

In [76]:
water_df.analysis_name.nunique()

39

In [77]:
water_df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,230,230,CO006WA0079,2012-07-19 00:00:00.000,Irrigation,sar,1.570,NaN
1,312,312,CP052WA0005,2012-07-13 00:00:00.000,Irrigation,sar,1.090,NaN
2,320,320,CP052WA0006,2012-07-13 00:00:00.000,Irrigation,sar,0.050,NaN
3,335,335,CP052WA0007,2012-07-13 00:00:00.000,Irrigation,sar,1.280,NaN
4,357,357,CP052WA0008,2012-07-13 00:00:00.000,Irrigation,sar,1.730,NaN
...,...,...,...,...,...,...,...,...
1006750,446831,446831,CU014WA0017,2020-04-09 10:34:00.000,KS Drinking Water Standard Analysis,nitrite,0.005,ppm
1006751,447540,447540,CU014WA0019,2020-04-09 11:01:00.000,KS Drinking Water Standard Analysis,nitrite,0.005,ppm
1006752,476832,476832,CB210WA0001,2020-05-12 10:49:00.000,KS Drinking Water Standard Analysis,nitrite,0.003,ppm
1006753,477818,477818,CB210WA0002,2020-05-12 10:49:00.000,KS Drinking Water Standard Analysis,nitrite,0.003,ppm


In [78]:
# water_df = water_df.loc[water_df['analysis_name'] == 'Base Titanium Water Analysis']
water_df

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
0,230,230,CO006WA0079,2012-07-19 00:00:00.000,Irrigation,sar,1.570,NaN
1,312,312,CP052WA0005,2012-07-13 00:00:00.000,Irrigation,sar,1.090,NaN
2,320,320,CP052WA0006,2012-07-13 00:00:00.000,Irrigation,sar,0.050,NaN
3,335,335,CP052WA0007,2012-07-13 00:00:00.000,Irrigation,sar,1.280,NaN
4,357,357,CP052WA0008,2012-07-13 00:00:00.000,Irrigation,sar,1.730,NaN
...,...,...,...,...,...,...,...,...
1006750,446831,446831,CU014WA0017,2020-04-09 10:34:00.000,KS Drinking Water Standard Analysis,nitrite,0.005,ppm
1006751,447540,447540,CU014WA0019,2020-04-09 11:01:00.000,KS Drinking Water Standard Analysis,nitrite,0.005,ppm
1006752,476832,476832,CB210WA0001,2020-05-12 10:49:00.000,KS Drinking Water Standard Analysis,nitrite,0.003,ppm
1006753,477818,477818,CB210WA0002,2020-05-12 10:49:00.000,KS Drinking Water Standard Analysis,nitrite,0.003,ppm


In [79]:
water_df.sort_values("batch_date")

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
739321,22111,22111,CM003WA0024,2010-08-25 00:00:00.000,Standard Drinking Water Analysis,fluorides,0.30,ppm
377993,22108,22108,CM003WA0024,2010-08-25 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,boron,0.01,ppm
464223,22125,22125,CM003WA0024,2010-08-25 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,manganese,0.05,ppm
729643,22117,22117,CM003WA0024,2010-08-25 00:00:00.000,Standard Drinking Water Analysis,calcium,9.42,ppm
488615,22116,22116,CM003WA0024,2010-08-25 00:00:00.000,Feacal Coliforms (MPN) and E.Coli,ammonium,0.01,ppm
...,...,...,...,...,...,...,...,...
932687,392769,392769,CS333WA0517,2024-08-07 11:37:22.000,Standard Drinking Water Analysis (WHO),silicon__,33.40,ppm
935714,398670,398670,CS333WA0517,2024-08-07 11:37:22.000,Standard Drinking Water Analysis (WHO),total_coliforms,70.00,mpn/100ml
931118,392966,392966,CS333WA0517,2024-08-07 11:37:22.000,Standard Drinking Water Analysis (WHO),ammonium,0.21,ppm
928797,392970,392970,CS333WA0517,2024-08-07 11:37:22.000,Standard Drinking Water Analysis (WHO),sodium,150.00,ppm


In [80]:
water_df = water_df.sort_values('sample_code')

In [81]:
new_samples_df = water_df.loc[water_df['batch_date'] > '2024-05-21'] 
water_df = water_df.loc[water_df['batch_date'] <= '2024-05-21'] 

In [82]:
water_df.analysis_name.unique()

array(['Irrigation', 'Reverse Osmosis Water Use',
       'Feacal Coliforms (MPN) and E.Coli', 'Standard Drinking Water',
       'Standard Drinking Water Analysis (WHO)',
       'Total Coliform and Faecal E.Coli', 'Heavy Metal Analysis (Water)',
       'Standard Drinking Water Analysis',
       'Basic Drinking Water Analysis', 'Heavy Metals in Water',
       'Total Dissolved Solids', 'Water Colour Analysis', 'Turbidity',
       'Total Suspended Solids', 'Basic Drinking Water Analysis (WHO)',
       'Feacal Coliform (MPN) and E.Coli', 'Post Harvest Water Analysis',
       'Total Bacterial Count', 'Total Coliforms and Faecal E.Coli',
       'Total Bacteria Count', 'Total Suspended Solids (Calculated)',
       'KEBS Drinking Water Microbiology', 'Chemical Oxygen Demand',
       'Chemical & Biochemical Oxygen Demand', 'Water Borne Diseases',
       'E coli & Total Coliforms', 'Water Microbiology (KEBS)',
       'KEBS Drinking Water Standard Analysis',
       'Colloidal Silica in Water', 'Ba

In [83]:
water_df['analysis_name'].nunique()

39

In [84]:
new_samples_df['analysis_name'].nunique()

6

In [85]:
new_samples_df[(new_samples_df['analysis_name'].isin(water_df['analysis_name'].unique()))].analysis_name.nunique()

6

In [86]:
water_df['analysis_name'] = [i.replace(":","") for i in water_df['analysis_name']]

In [87]:
water_df['analysis_name'].value_counts()

Irrigation                                          374744
Feacal Coliforms (MPN) and E.Coli                   136524
Basic Drinking Water Analysis                       121173
Standard Drinking Water Analysis                    111316
Standard Drinking Water                              63749
Reverse Osmosis Water Use                            44505
Base Titanium Water Analysis                         35185
Total Coliform and Faecal E.Coli                     28720
Standard Drinking Water Analysis (WHO)               19280
Heavy Metal Analysis (Water)                         16642
Total Dissolved Solids                               11430
Taita Water Analysis (Uni of Helsinki)                8256
Water Colour Analysis                                 7449
Heavy Metals in Water                                 6250
Feacal Coliform (MPN) and E.Coli                      4776
Total Suspended Solids (Calculated)                   2871
Turbidity                                             28

In [88]:
for analysis in np.unique(water_df['analysis_name']):
    print(analysis)
    df_ = water_df.loc[water_df['analysis_name']==analysis]
    df_ = pd.pivot_table(data=df_, values="result", index="sample_code", columns="chemical_name")
    print(len(df_))

    os.makedirs(f"output/boxplots/{analysis}",exist_ok=True)
    for column in df_.columns:
        plt.boxplot(df_[column])
        plt.savefig(f"output/boxplots/{analysis}/{column}.png")
        plt.clf()
        outlier_threshold = df_[column].quantile(0.99)
        # df_ = df_.loc[df_[column] <= outlier_threshold]
    df_.describe().to_csv(f"output/chemical_null_count/{analysis}.csv")
    print(len(df_))
    if(len(df_) == 0):
        continue
    
    df_.to_csv(f"output/analysis/{analysis}.csv")
    

Aquasearch Full Chemical Analysis
191
191
BT Extended Water Quality Analysis
28
28
Base Titanium Water Analysis
1548
1548
Basic Drinking Water Analysis
2720
2720
Basic Drinking Water Analysis (WHO)
338
338
Chemical & Biochemical Oxygen Demand
160
160
Chemical Oxygen Demand
79
79
Colloidal Silica in Water
30
30
E coli & Total Coliforms
105
105
Feacal Coliform (MPN) and E.Coli
214
214
Feacal Coliforms (MPN) and E.Coli
3344
3344
Free Chlorine
77
77
Heavy Metal Analysis (Water)
624
624
Heavy Metals in Water
654
654
Irrigation
6632
6632
KEBS Drinking Water Microbiology
71
71
KEBS Drinking Water Standard Analysis
94
94
KS Drinking Water Standard Analysis
19
19
Oil & Grease
63
63
Post Harvest Water Analysis
121
121
Reverse Osmosis Water Use
1785
1785
Standard Drinking Water
1420
1420
Standard Drinking Water + Free Chlorine Analysis
43
43
Standard Drinking Water Analysis
2473
2473
Standard Drinking Water Analysis (WHO)
907
907
Taita Water Analysis (Uni of Helsinki)
516
516
Total Bacteria Count

PermissionError: [Errno 13] Permission denied: 'output/analysis/Total Bacterial Count.csv'

<Figure size 640x480 with 0 Axes>

In [18]:
# for analysis_file in os.listdir("output/analysis"):
#     os.makedirs("./output/pairplots",exist_ok=True)
#     analysis_df = pd.read_csv(f"output/analysis/{analysis_file}",index_col=0)
#     print(analysis_file.replace('.csv',''))
#     sns.pairplot(analysis_df)
#     print('Saving')
#     plt.savefig(f"output/pairplots/{analysis_file.replace('.csv','')}.png")
#     print('Saved')
#     plt.clf()

In [19]:
for analysis_file in os.listdir("output/analysis"):
    print(analysis_file)
    analysis_df = pd.read_csv(f"output/analysis/{analysis_file}",index_col=0)
    print(len(analysis_df.columns))
    imp_mean = SimpleImputer(missing_values=np.nan, strategy='mean')
    analysis_df = pd.DataFrame(imp_mean.fit_transform(analysis_df), columns=analysis_df.columns, index=analysis_df.index)
    pickle.dump(imp_mean, open(f"models/imputers/{analysis_file.replace('csv','pkl')}","wb"))
    # if(len(analysis_df) == 0):
    #     continue
    sc = StandardScaler()
    pca = PCA()
    analysis_scaled = sc.fit_transform(analysis_df)
    pca.fit(analysis_scaled)
    pca_explained_variance_df = pd.DataFrame(zip(analysis_df.columns,pca.explained_variance_ratio_))
    pca_explained_variance_df.to_csv(f"output/pca_explained_variance_ratio/{analysis_file}")
    pca_explained_variance_df = pca_explained_variance_df.loc[pca_explained_variance_df[1]>0.1]
    n_components = len(pca_explained_variance_df)
    
    pca_chems = pca_explained_variance_df[0]
    
    pca = PCA(n_components=n_components)
    analysis_scaled = sc.fit_transform(analysis_df)
    pca_reduced_df = pca.fit_transform(analysis_scaled)
    analysis_scaled = pd.DataFrame(analysis_scaled,index=analysis_df.index)
    pca_reduced_df = pd.DataFrame(pca_reduced_df,index=analysis_df.index, columns=pca_chems)
    analysis_scaled.to_csv(f"output/analysis_scaled/{analysis_file}")
    pca_reduced_df.to_csv(f"output/pca_df/{analysis_file}")
    pickle.dump(sc, open(f"models/scalers/{analysis_file.replace('csv','pkl')}","wb"))
    pickle.dump(pca, open(f"models/pca/{analysis_file.replace('csv','pkl')}","wb"))

Aquasearch Full Chemical Analysis.csv
4
Base Titanium Water Analysis.csv
23
Basic Drinking Water Analysis (WHO).csv
2
Basic Drinking Water Analysis.csv
43
BT Extended Water Quality Analysis.csv
35
Chemical & Biochemical Oxygen Demand.csv
20
Chemical Oxygen Demand.csv
3
Colloidal Silica in Water.csv
3
E coli & Total Coliforms.csv
2
Feacal Coliform (MPN) and E.Coli.csv
26
Feacal Coliforms (MPN) and E.Coli.csv
37
Free Chlorine.csv
5
Heavy Metal Analysis (Water).csv
35
Heavy Metals in Water.csv
15
Irrigation.csv
41
KEBS Drinking Water Microbiology.csv
6
KEBS Drinking Water Standard Analysis.csv
4
KS Drinking Water Standard Analysis.csv
3
Oil & Grease.csv
8
Post Harvest Water Analysis.csv
3
Reverse Osmosis Water Use.csv
35
Standard Drinking Water + Free Chlorine Analysis.csv
2
Standard Drinking Water Analysis (WHO).csv
25
Standard Drinking Water Analysis.csv
31
Standard Drinking Water.csv
26
Taita Water Analysis (Uni of Helsinki).csv
16
Total Bacteria Count.csv
8
Total Bacterial Count.csv
4

In [20]:
# pd.DataFrame([
#     {
#         'sample_code': 'CB057WA0107',
#         'ammonium': {'result':3.936512250255515, 'units':'ppm'},
#         'bicarbonate': {'result':3.936512250255515, 'units':'ppm'},
#         'boron': {'result':3.936512250255515, 'units':'ppm'}
#     }
# ])

In [21]:
# pca_reduced_df.reset_index().to_dict('records')

In [22]:
for analysis_file in os.listdir("output/pca"):
    print(analysis_file)

Aquasearch Full Chemical Analysis.csv
Base Titanium Water Analysis.csv
Basic Drinking Water Analysis (WHO).csv
Basic Drinking Water Analysis.csv
BT Extended Water Quality Analysis.csv
Chemical & Biochemical Oxygen Demand.csv
Chemical Oxygen Demand.csv
Colloidal Silica in Water.csv
E coli & Total Coliforms.csv
Feacal Coliform (MPN) and E.Coli.csv
Feacal Coliforms (MPN) and E.Coli.csv
Free Chlorine.csv
Heavy Metal Analysis (Water).csv
Heavy Metals in Water.csv
Irrigation.csv
KEBS Drinking Water Microbiology.csv
KEBS Drinking Water Standard Analysis.csv
KS Drinking Water Standard Analysis.csv
Oil & Grease.csv
Post Harvest Water Analysis.csv
Reverse Osmosis Water Use.csv
Standard Drinking Water + Free Chlorine Analysis.csv
Standard Drinking Water Analysis (WHO).csv
Standard Drinking Water Analysis.csv
Standard Drinking Water.csv
Taita Water Analysis (Uni of Helsinki).csv
Total Bacteria Count.csv
Total Bacterial Count.csv
Total Chlorine.csv
Total Coliform and Faecal E.Coli.csv
Total Colifor

In [23]:
for analysis_file in os.listdir("output/pca_df"):
    print(analysis_file)
    pca_reduced_df = pd.read_csv(f"output/pca_df/{analysis_file}",index_col=0)
    # if len(pca_reduced_df.columns) < 2:
        # continue
    mu = np.mean(pca_reduced_df, axis=0)
    sigma = np.cov(pca_reduced_df.T)

    try:
        pca_reduced_df['mahalanobis_distance'] = [distance.mahalanobis(pca_reduced_df.iloc[i], mu, np.linalg.inv(sigma)) for i in range(len(pca_reduced_df)) ]
    except:
        continue
    pca_reduced_df.to_csv(f"output/mahalanobis_distance/{analysis_file}")

Aquasearch Full Chemical Analysis.csv
Base Titanium Water Analysis.csv
Basic Drinking Water Analysis (WHO).csv
Basic Drinking Water Analysis.csv
BT Extended Water Quality Analysis.csv
Chemical & Biochemical Oxygen Demand.csv
Chemical Oxygen Demand.csv
Colloidal Silica in Water.csv
E coli & Total Coliforms.csv
Feacal Coliform (MPN) and E.Coli.csv
Feacal Coliforms (MPN) and E.Coli.csv
Free Chlorine.csv
Heavy Metal Analysis (Water).csv
Heavy Metals in Water.csv
Irrigation.csv
KEBS Drinking Water Microbiology.csv
KEBS Drinking Water Standard Analysis.csv
KS Drinking Water Standard Analysis.csv
Oil & Grease.csv
Post Harvest Water Analysis.csv
Reverse Osmosis Water Use.csv
Standard Drinking Water + Free Chlorine Analysis.csv
Standard Drinking Water Analysis (WHO).csv
Standard Drinking Water Analysis.csv
Standard Drinking Water.csv
Taita Water Analysis (Uni of Helsinki).csv
Total Bacteria Count.csv
Total Bacterial Count.csv
Total Chlorine.csv
Total Coliform and Faecal E.Coli.csv
Total Colifor

In [24]:
pca_reduced_df

,total_viable_count_@37
sample_code,
CA361WA0008,-0.814164
CA361WA0009,-0.778229
CA361WA0010,0.736775
CA361WA0011,2.059170
CA361WA0012,0.248064
CA361WA0013,-0.726483
CB050WA0037,-0.805252
CB050WA0038,-0.694861
CB050WA0039,-0.594244


In [25]:
mahalanobis_threshold_dict = {}
for analysis_file in os.listdir("output/mahalanobis_distance"):
    print(analysis_file)
    analysis = analysis_file.replace(".csv","")
    mahalanobis_df = pd.read_csv(f"output/mahalanobis_distance/{analysis_file}",index_col=0)
    upper_quantile = (mahalanobis_df['mahalanobis_distance'].quantile(0.95))
    mahalanobis_threshold_dict[analysis_file.replace(".csv","")] = upper_quantile
    mahalanobis_df.loc[mahalanobis_df['mahalanobis_distance'] >= upper_quantile].to_csv(f"output/mahanobis_distance_upper_quantile/{analysis_file}")
    plt.boxplot(mahalanobis_df['mahalanobis_distance'])
    plt.savefig(f"output/mahalanobis_boxplots/{analysis}.png")
    plt.clf()
pickle.dump(mahalanobis_threshold_dict, open("mahalanobis_thresholds.dict","wb"))

Aquasearch Full Chemical Analysis.csv
Base Titanium Water Analysis.csv
Basic Drinking Water Analysis (WHO).csv
BT Extended Water Quality Analysis.csv
Chemical & Biochemical Oxygen Demand.csv
Chemical Oxygen Demand.csv
Colloidal Silica in Water.csv
E coli & Total Coliforms.csv
Feacal Coliform (MPN) and E.Coli.csv
Free Chlorine.csv
Heavy Metal Analysis (Water).csv
Heavy Metals in Water.csv
Irrigation.csv
KEBS Drinking Water Microbiology.csv
KEBS Drinking Water Standard Analysis.csv
KS Drinking Water Standard Analysis.csv
Oil & Grease.csv
Reverse Osmosis Water Use.csv
Standard Drinking Water + Free Chlorine Analysis.csv
Standard Drinking Water Analysis.csv
Standard Drinking Water.csv
Taita Water Analysis (Uni of Helsinki).csv
Total Bacteria Count.csv
Total Bacterial Count.csv
Total Chlorine.csv
Total Coliform and Faecal E.Coli.csv
Total Coliforms and Faecal E.Coli.csv
Total Suspended Solids (Calculated).csv
Total Suspended Solids.csv
Turbidity Analysis.csv
Turbidity.csv
Water Borne Diseas

<Figure size 640x480 with 0 Axes>

In [26]:
len(mahalanobis_threshold_dict.keys())

33

In [27]:
new_samples_df.analysis_name.unique()

array(['Standard Drinking Water Analysis (WHO)',
       'KEBS Drinking Water Microbiology',
       'Basic Drinking Water Analysis (WHO)', 'Irrigation',
       'Standard Drinking Water + Free Chlorine Analysis',
       'Reverse Osmosis Water Use'], dtype=object)

In [30]:
conn_lims = pyodbc.connect("Driver={SQL Server};"
                            "Server=192.168.5.18\CROPNUT;"
                            "Database=cropnuts;"
                            "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [31]:
reverse_analysis_dict = pd.read_sql(f"""SELECT analysis_id, LTRIM(RTRIM(analysis_name)) as analysis_name FROM Analysis ORDER BY analysis_name""",con=conn_lims).set_index("analysis_name").to_dict()['analysis_id']
reverse_analysis_dict

C:\Users\tommy\AppData\Roaming\Python\Python38\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


{'% Gypsum': 540,
 '% Nitrogen (RSSP 2)': 647,
 '% Organic Matter': 258,
 '% Organic Matter (RSSP 2)': 646,
 '% Soil Nitrogen': 18,
 '%Assay': 27235,
 '%N (RSSP 2)': 652,
 '%OM (RSSP 2)': 651,
 '%P,%S Analysis(Super Calcium)': 383,
 '1:2 Soil Volume Extract': 294,
 '1:2 soil volume extract': 25,
 '1:2 Soil Volume Extract (Data Only)': 27339,
 '1:2 Vol Extract for BLGG': 317,
 '20:12:12 (N,P205,K20) Analysis': 840,
 'Absorbance at 254nm': 26936,
 'Acid Detergent Fibre': 27063,
 'Acid Insoluble Ash': 27417,
 'Acid Insoluble Matter': 27108,
 'Acid titration': 26647,
 'Acid Value': 26971,
 'Advanced Biological Farming Soil Audit': 26521,
 'Advanced Soil Health Analysis': 27181,
 'Aerobic Mesophilic Count': 26649,
 'Aflatoxin': 26794,
 'Aflatoxin AFB1': 26938,
 'Aflatoxin B1': 26939,
 'Aflatoxin in feed': 767,
 'Aflatoxin Total': 26753,
 'AfSIS Standard Leaf Analysis': 633,
 'AFSIS Std Wet Chemistry Soil Analysis': 589,
 'Aggregate Stability': 26727,
 'Agrifi Soil Microbiome Analysis': 2729

In [32]:
len(reverse_analysis_dict.keys())

1323

In [33]:
test = []

In [34]:
for sample in new_samples_df.sample_code.unique():
    res = {}
    tmp_ = new_samples_df.loc[new_samples_df.sample_code == sample]
    res['sample_code'] = sample
    print(tmp_.analysis_name.unique())
    res['analysis_id'] = [ reverse_analysis_dict[i] for i in tmp_.analysis_name.unique() if i in reverse_analysis_dict.keys() ]
    for index,row in tmp_.iterrows():
        if row['result'] >= 0 :
            res[row['chemical_name']] = {'result': row['result'], 'units': str(row['unit_name'])}
        else:
            res[row['chemical_name']] = {'result': 0, 'units': row['unit_name']}
    test.append(res)    


['Standard Drinking Water Analysis (WHO)']
['Standard Drinking Water Analysis (WHO)']
['Standard Drinking Water Analysis (WHO)']
['Standard Drinking Water Analysis (WHO)']
['KEBS Drinking Water Microbiology']
['Standard Drinking Water Analysis (WHO)']
['Standard Drinking Water Analysis (WHO)']
['Standard Drinking Water Analysis (WHO)']
['Standard Drinking Water Analysis (WHO)']
['Standard Drinking Water Analysis (WHO)']
['Standard Drinking Water Analysis (WHO)']
['Standard Drinking Water Analysis (WHO)']
['Standard Drinking Water Analysis (WHO)']
['Standard Drinking Water Analysis (WHO)']
['Standard Drinking Water Analysis (WHO)']
['Basic Drinking Water Analysis (WHO)' 'KEBS Drinking Water Microbiology']
['Standard Drinking Water Analysis (WHO)']
['Irrigation']
['Standard Drinking Water Analysis (WHO)']
['Standard Drinking Water + Free Chlorine Analysis']
['Standard Drinking Water Analysis (WHO)']
['Irrigation']
['Irrigation']
['Reverse Osmosis Water Use']
['Standard Drinking Water Ana

In [35]:
# test

In [32]:
str(test).replace("'", '"')

'[{"sample_code": "CA028WA0129", "analysis_id": [707], "potassium": {"result": 11.4, "units": "ppm"}, "magnesium": {"result": 8.36, "units": "ppm"}, "calcium": {"result": 14.8, "units": "ppm"}, "sodium": {"result": 40.2, "units": "ppm"}, "electrical_conductivity": {"result": 0.34, "units": "mS cm -1"}, "sulphur": {"result": 4.61, "units": "ppm"}, "phosphorus": {"result": 0.1, "units": "ppm"}, "ammonium": {"result": 0.01, "units": "ppm"}, "silicon__": {"result": 28.7, "units": "ppm"}, "nitrate_n": {"result": 6.03, "units": "ppm"}, "silica": {"result": 61.4, "units": "ppm"}}, {"sample_code": "CA028WA0130", "analysis_id": [707], "hardness": {"result": 68.6, "units": "ppm"}, "silica": {"result": 58.8, "units": "ppm"}, "phosphorus": {"result": 0.093, "units": "ppm"}, "zinc": {"result": 0.01, "units": "ppm"}, "calcium": {"result": 14.4, "units": "ppm"}, "sulphate": {"result": 12.1, "units": "ppm"}, "bicarbonate": {"result": 132.0, "units": "ppm"}, "sulphur": {"result": 4.03, "units": "ppm"},

In [36]:
reverse_analysis_dict[ 'KEBS Drinking Water Microbiology']

26724

In [37]:
new_samples_df.analysis_name.unique()

array(['Standard Drinking Water Analysis (WHO)',
       'KEBS Drinking Water Microbiology',
       'Basic Drinking Water Analysis (WHO)', 'Irrigation',
       'Standard Drinking Water + Free Chlorine Analysis',
       'Reverse Osmosis Water Use'], dtype=object)

In [38]:
mahalanobis_threshold_dict['Standard Drinking Water Analysis (WHO)']

KeyError: 'Standard Drinking Water Analysis (WHO)'

In [39]:
mahalanobis_threshold_dict.keys()

dict_keys(['Aquasearch Full Chemical Analysis', 'Base Titanium Water Analysis', 'Basic Drinking Water Analysis (WHO)', 'BT Extended Water Quality Analysis', 'Chemical & Biochemical Oxygen Demand', 'Chemical Oxygen Demand', 'Colloidal Silica in Water', 'E coli & Total Coliforms', 'Feacal Coliform (MPN) and E.Coli', 'Free Chlorine', 'Heavy Metal Analysis (Water)', 'Heavy Metals in Water', 'Irrigation', 'KEBS Drinking Water Microbiology', 'KEBS Drinking Water Standard Analysis', 'KS Drinking Water Standard Analysis', 'Oil & Grease', 'Reverse Osmosis Water Use', 'Standard Drinking Water + Free Chlorine Analysis', 'Standard Drinking Water Analysis', 'Standard Drinking Water', 'Taita Water Analysis (Uni of Helsinki)', 'Total Bacteria Count', 'Total Bacterial Count', 'Total Chlorine', 'Total Coliform and Faecal E.Coli', 'Total Coliforms and Faecal E.Coli', 'Total Suspended Solids (Calculated)', 'Total Suspended Solids', 'Turbidity Analysis', 'Turbidity', 'Water Borne Diseases', 'Water Colour 

In [41]:
water_ = new_samples_df.loc[new_samples_df['analysis_name'].isin(mahalanobis_threshold_dict.keys())]

In [42]:
water_

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
1005615,267276,267276,CA373WA0077,2024-06-21 14:10:00.000,KEBS Drinking Water Microbiology,electrical_conductivity,0.18,mS cm -1
980118,271966,271966,CD048WA0110,2024-06-21 12:41:00.000,Basic Drinking Water Analysis (WHO),calcium,13.40,ppm
1005616,271964,271964,CD048WA0110,2024-06-21 12:41:00.000,KEBS Drinking Water Microbiology,electrical_conductivity,0.28,mS cm -1
980449,271967,271967,CD048WA0110,2024-06-21 12:41:00.000,Basic Drinking Water Analysis (WHO),magnesium,3.90,ppm
106687,387408,387408,CE021WA0022,2024-05-21 12:38:28.263,Irrigation,silicon__,5.79,ppm
...,...,...,...,...,...,...,...,...
980124,278510,278510,CT426WA0005,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),calcium,20.80,ppm
980133,296876,296876,CT426WA0006,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),calcium,136.00,ppm
980466,296877,296877,CT426WA0006,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),magnesium,101.00,ppm
980467,296883,296883,CT426WA0007,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),magnesium,8.12,ppm


In [43]:
water_ = water_.sort_values(by="sample_code")

In [44]:
# water_ = water_.loc[(water_['sample_code'] == 'CD048WA0110')]

In [45]:
water_

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
1005615,267276,267276,CA373WA0077,2024-06-21 14:10:00.000,KEBS Drinking Water Microbiology,electrical_conductivity,0.18,mS cm -1
980118,271966,271966,CD048WA0110,2024-06-21 12:41:00.000,Basic Drinking Water Analysis (WHO),calcium,13.40,ppm
1005616,271964,271964,CD048WA0110,2024-06-21 12:41:00.000,KEBS Drinking Water Microbiology,electrical_conductivity,0.28,mS cm -1
980449,271967,271967,CD048WA0110,2024-06-21 12:41:00.000,Basic Drinking Water Analysis (WHO),magnesium,3.90,ppm
68410,378300,378300,CE021WA0022,2024-05-21 12:38:28.263,Irrigation,ammonium,0.01,ppm
...,...,...,...,...,...,...,...,...
980124,278510,278510,CT426WA0005,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),calcium,20.80,ppm
980133,296876,296876,CT426WA0006,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),calcium,136.00,ppm
980466,296877,296877,CT426WA0006,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),magnesium,101.00,ppm
980467,296883,296883,CT426WA0007,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),magnesium,8.12,ppm


In [46]:
water_pivot = pd.pivot_table(data = water_, columns = "chemical_name", values = "result", index="sample_code").reset_index()

In [47]:
water_

,Unnamed: 0.1,Unnamed: 0,sample_code,batch_date,analysis_name,chemical_name,result,unit_name
1005615,267276,267276,CA373WA0077,2024-06-21 14:10:00.000,KEBS Drinking Water Microbiology,electrical_conductivity,0.18,mS cm -1
980118,271966,271966,CD048WA0110,2024-06-21 12:41:00.000,Basic Drinking Water Analysis (WHO),calcium,13.40,ppm
1005616,271964,271964,CD048WA0110,2024-06-21 12:41:00.000,KEBS Drinking Water Microbiology,electrical_conductivity,0.28,mS cm -1
980449,271967,271967,CD048WA0110,2024-06-21 12:41:00.000,Basic Drinking Water Analysis (WHO),magnesium,3.90,ppm
68410,378300,378300,CE021WA0022,2024-05-21 12:38:28.263,Irrigation,ammonium,0.01,ppm
...,...,...,...,...,...,...,...,...
980124,278510,278510,CT426WA0005,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),calcium,20.80,ppm
980133,296876,296876,CT426WA0006,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),calcium,136.00,ppm
980466,296877,296877,CT426WA0006,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),magnesium,101.00,ppm
980467,296883,296883,CT426WA0007,2024-07-19 16:11:00.000,Basic Drinking Water Analysis (WHO),magnesium,8.12,ppm


In [48]:
water_pivot['analysis_id'] = 570

In [49]:
req = water_pivot.to_dict(orient="records")

In [50]:
req

[{'sample_code': 'CA373WA0077',
  'ammonium': nan,
  'bicarbonate': nan,
  'boron': nan,
  'calcium': nan,
  'chloride': nan,
  'copper': nan,
  'electrical_conductivity': 0.18,
  'faecal_e_coli': nan,
  'hardness': nan,
  'iron': nan,
  'magnesium': nan,
  'manganese': nan,
  'molybdenum': nan,
  'nitrate_n': nan,
  'nitrates': nan,
  'phosphorus': nan,
  'potassium': nan,
  'sar': nan,
  'silicon__': nan,
  'sodium': nan,
  'sulphate': nan,
  'sulphur': nan,
  'zinc': nan,
  'analysis_id': 570},
 {'sample_code': 'CD048WA0110',
  'ammonium': nan,
  'bicarbonate': nan,
  'boron': nan,
  'calcium': 13.4,
  'chloride': nan,
  'copper': nan,
  'electrical_conductivity': 0.28,
  'faecal_e_coli': nan,
  'hardness': nan,
  'iron': nan,
  'magnesium': 3.9,
  'manganese': nan,
  'molybdenum': nan,
  'nitrate_n': nan,
  'nitrates': nan,
  'phosphorus': nan,
  'potassium': nan,
  'sar': nan,
  'silicon__': nan,
  'sodium': nan,
  'sulphate': nan,
  'sulphur': nan,
  'zinc': nan,
  'analysis_id':

In [51]:
import psycopg2

conn_lims = pyodbc.connect("Driver={SQL Server};"
                            "Server=192.168.5.18\CROPNUT;"
                            "Database=cropnuts;"
                            "uid=thomasTsuma;pwd=GR^KX$uRe9#JwLc6")

In [52]:
analysis_dict = pd.read_sql(f"""SELECT analysis_id, LTRIM(RTRIM(analysis_name)) as analysis_name FROM Analysis  ORDER BY analysis_name""",con=conn_lims).set_index("analysis_id").to_dict()['analysis_name']


C:\Users\tommy\AppData\Roaming\Python\Python38\site-packages\pandas\io\sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [53]:
analysis_dict

{540: '% Gypsum',
 647: '% Nitrogen (RSSP 2)',
 258: '% Organic Matter',
 646: '% Organic Matter (RSSP 2)',
 18: '% Soil Nitrogen',
 27235: '%Assay',
 652: '%N (RSSP 2)',
 651: '%OM (RSSP 2)',
 383: '%P,%S Analysis(Super Calcium)',
 294: '1:2 Soil Volume Extract',
 25: '1:2 soil volume extract',
 27339: '1:2 Soil Volume Extract (Data Only)',
 317: '1:2 Vol Extract for BLGG',
 840: '20:12:12 (N,P205,K20) Analysis',
 26936: 'Absorbance at 254nm',
 27063: 'Acid Detergent Fibre',
 27146: 'Acid Insoluble Ash',
 27003: 'Acid Insoluble Ash',
 27417: 'Acid Insoluble Ash',
 27108: 'Acid Insoluble Matter',
 26647: 'Acid titration',
 26971: 'Acid Value',
 26521: 'Advanced Biological Farming Soil Audit',
 27181: 'Advanced Soil Health Analysis',
 26649: 'Aerobic Mesophilic Count',
 26794: 'Aflatoxin',
 26938: 'Aflatoxin AFB1',
 26939: 'Aflatoxin B1',
 767: 'Aflatoxin in feed',
 766: 'Aflatoxin Total',
 26753: 'Aflatoxin Total',
 633: 'AfSIS Standard Leaf Analysis',
 589: 'AFSIS Std Wet Chemistry So

In [54]:
pickle.dump(analysis_dict, open("analysis.dict","wb"))

In [55]:
req = [
    {
      'sample_code': 'CD048WA0110',
      'calcium': {'result':13.4, "units":"ppm"},
      'electrical_conductivity': {"result":0.28, "units":"mS cm -1"},
      'magnesium': {'result':3.9, "units":"ppm"},
      'analysis_id': [288,27032]
    }]


In [56]:
req

[{'sample_code': 'CD048WA0110',
  'calcium': {'result': 13.4, 'units': 'ppm'},
  'electrical_conductivity': {'result': 0.28, 'units': 'mS cm -1'},
  'magnesium': {'result': 3.9, 'units': 'ppm'},
  'analysis_id': [288, 27032]}]

In [57]:
_ = pd.DataFrame(req)

In [58]:
_

,sample_code,calcium,electrical_conductivity,magnesium,analysis_id
0,CD048WA0110,"{'result': 13.4, 'units': 'ppm'}","{'result': 0.28, 'units': 'mS cm -1'}","{'result': 3.9, 'units': 'ppm'}","[288, 27032]"


In [59]:
analysis_dict = pickle.load(open("analysis.dict","rb"))

In [60]:
analysis_dict

{540: '% Gypsum',
 647: '% Nitrogen (RSSP 2)',
 258: '% Organic Matter',
 646: '% Organic Matter (RSSP 2)',
 18: '% Soil Nitrogen',
 27235: '%Assay',
 652: '%N (RSSP 2)',
 651: '%OM (RSSP 2)',
 383: '%P,%S Analysis(Super Calcium)',
 294: '1:2 Soil Volume Extract',
 25: '1:2 soil volume extract',
 27339: '1:2 Soil Volume Extract (Data Only)',
 317: '1:2 Vol Extract for BLGG',
 840: '20:12:12 (N,P205,K20) Analysis',
 26936: 'Absorbance at 254nm',
 27063: 'Acid Detergent Fibre',
 27146: 'Acid Insoluble Ash',
 27003: 'Acid Insoluble Ash',
 27417: 'Acid Insoluble Ash',
 27108: 'Acid Insoluble Matter',
 26647: 'Acid titration',
 26971: 'Acid Value',
 26521: 'Advanced Biological Farming Soil Audit',
 27181: 'Advanced Soil Health Analysis',
 26649: 'Aerobic Mesophilic Count',
 26794: 'Aflatoxin',
 26938: 'Aflatoxin AFB1',
 26939: 'Aflatoxin B1',
 767: 'Aflatoxin in feed',
 766: 'Aflatoxin Total',
 26753: 'Aflatoxin Total',
 633: 'AfSIS Standard Leaf Analysis',
 589: 'AFSIS Std Wet Chemistry So

In [61]:
mahalanobis_thresholds = pickle.load(open("mahalanobis_thresholds.dict","rb"))

In [62]:
(mahalanobis_thresholds.keys())

dict_keys(['Aquasearch Full Chemical Analysis', 'Base Titanium Water Analysis', 'Basic Drinking Water Analysis (WHO)', 'BT Extended Water Quality Analysis', 'Chemical & Biochemical Oxygen Demand', 'Chemical Oxygen Demand', 'Colloidal Silica in Water', 'E coli & Total Coliforms', 'Feacal Coliform (MPN) and E.Coli', 'Free Chlorine', 'Heavy Metal Analysis (Water)', 'Heavy Metals in Water', 'Irrigation', 'KEBS Drinking Water Microbiology', 'KEBS Drinking Water Standard Analysis', 'KS Drinking Water Standard Analysis', 'Oil & Grease', 'Reverse Osmosis Water Use', 'Standard Drinking Water + Free Chlorine Analysis', 'Standard Drinking Water Analysis', 'Standard Drinking Water', 'Taita Water Analysis (Uni of Helsinki)', 'Total Bacteria Count', 'Total Bacterial Count', 'Total Chlorine', 'Total Coliform and Faecal E.Coli', 'Total Coliforms and Faecal E.Coli', 'Total Suspended Solids (Calculated)', 'Total Suspended Solids', 'Turbidity Analysis', 'Turbidity', 'Water Borne Diseases', 'Water Colour 

In [63]:
unit_decision = pd.read_csv("output/water_unit_per_chemical_decision.csv")

In [64]:
unit_decision

,Unnamed: 0,crop,chemical_name,unit_name
0,0,Irrigation,sar,NaN
1,0,Irrigation,nitrate_n,ppm
2,0,Irrigation,potassium,ppm
3,0,Irrigation,sulphur,ppm
4,0,Irrigation,phosphorus,ppm
...,...,...,...,...
832,0,Colloidal Silica in Water,colloidal_silica,ppm
833,0,Water Microbiology (KEBS),total_viable_count_@37,mpn/ml
834,0,KS Drinking Water Standard Analysis,total_coliforms,cfu/ml
835,0,KS Drinking Water Standard Analysis,faecal_e_coli,cfu/ml


In [105]:
req_body = test
_ = pd.DataFrame(req_body)
unit_decision = pd.read_csv("water_unit_per_chemical_decision.csv")
analysis_dict = pickle.load(open("analysis.dict","rb"))
mahalanobis_thresholds = pickle.load(open("mahalanobis_thresholds.dict","rb"))

result = {}
_df = pd.DataFrame()
for index, row in _.iterrows():
    analyses = row['analysis_id']
    for analysis_ in analyses:
        df_analysis_ = pd.DataFrame(row).T
        df_analysis_['analysis_name'] = analysis_dict[analysis_]
        _df = pd.concat([_df, df_analysis_])
for index,row in _df.iterrows():
    sample_code = row['sample_code']
    analysis = row['analysis_name']
    analysis_id = row['analysis_id']
    if sample_code not in result.keys():
        result[sample_code] = []
    if analysis not in mahalanobis_thresholds.keys():
        result[sample_code].append({"sample_code": sample_code,"status":"warning", "message": f"Analysis not in models", "details": f"Analysis: {analysis} is not in the list of defined models" })   
        continue     
    scaler = pickle.load(open(f"scalers/{analysis}.pkl","rb"))
    pca = pickle.load(open(f"pca/{analysis}.pkl","rb"))

    pca_df = pd.read_csv(f"pca_df/{analysis}.csv",index_col=0)
    analysis_df = pd.read_csv(f"analysis/{analysis}.csv",index_col=0)

    

    try :
        tmp_df = pd.DataFrame(row).T[analysis_df.columns]
        # tmp_df = tmp_df.dropna() 
    except:
        result[sample_code].append({"sample_code": sample_code,"status":"warning", "message": f"Missing parameters for analysis_id: {analysis}", "details": f"Expected parameters are {','.join(pca_df.columns)} for analysis: {analysis}" })
        continue
    failed_units_comparison = {}    
    for col in pca_df.columns:
        expected_units = unit_decision.loc[(unit_decision['crop'] == analysis) & (unit_decision['chemical_name'] == col)]
        print(expected_units[['crop','chemical_name','unit_name']].to_dict())
        import math
        if type(row[col]) != dict and math.isnan(row[col]):
            failed_units_comparison[col] = expected_units[['crop','chemical_name','unit_name']].to_dict()
        elif row[col]['units'] !=   expected_units['unit_name'].values[0] :
            failed_units_comparison[col] = expected_units[['crop','chemical_name','unit_name']].to_dict()
        else:
            row[col] = row[col]['result']
    if len(failed_units_comparison.keys()) > 0:
        result[sample_code].append({"sample_code": sample_code,"status":"warning", "message": f"Wrong units provided", "details": f"Expected units are {str(failed_units_comparison)} for analysis: {analysis}" })
        continue

    tmp_df = pd.DataFrame(row).T[analysis_df.columns]
    print(tmp_df.columns)
    # tmp_df = 
    df_scaled = scaler.transform(tmp_df)
    df_pca = pd.DataFrame(pca.transform(df_scaled))

    mu = np.mean(pca_df, axis=0)
    sigma = np.cov(pca_df.T)

    mahalanobis_distance = distance.mahalanobis(df_pca.iloc[0], mu, np.linalg.inv(sigma))

    print(mahalanobis_distance)

    expected_md = mahalanobis_thresholds[analysis]
    print(expected_md)

    if mahalanobis_distance > expected_md:
        result[sample_code].append({"sample_code": sample_code,"status":"fail", "message": "Mahalanobis distance exceeds threshold", "description":f"Mahalanobis distance of {mahalanobis_distance} exceeds threshold of {expected_md} for analysis: {analysis}" })
    else:
        result[sample_code].append({"sample_code": sample_code,"status":"pass","message": "Mahalanobis distance within threshold", "description":f"Mahalanobis distance of {mahalanobis_distance} is within threshold of {expected_md} for analysis: {analysis}" })
            


{'crop': {549: 'Basic Drinking Water Analysis (WHO)'}, 'chemical_name': {549: 'calcium'}, 'unit_name': {549: 'ppm'}}
{'crop': {550: 'Basic Drinking Water Analysis (WHO)'}, 'chemical_name': {550: 'magnesium'}, 'unit_name': {550: 'ppm'}}
Index(['calcium', 'magnesium'], dtype='object')
0.3319501004886362
2.179297665026459
{'crop': {632: 'Standard Drinking Water + Free Chlorine Analysis'}, 'chemical_name': {632: 'faecal_e_coli'}, 'unit_name': {632: 'mpn/100ml'}}
{'crop': {631: 'Standard Drinking Water + Free Chlorine Analysis'}, 'chemical_name': {631: 'total_coliforms'}, 'unit_name': {631: 'mpn/100ml'}}
{'crop': {549: 'Basic Drinking Water Analysis (WHO)'}, 'chemical_name': {549: 'calcium'}, 'unit_name': {549: 'ppm'}}
{'crop': {550: 'Basic Drinking Water Analysis (WHO)'}, 'chemical_name': {550: 'magnesium'}, 'unit_name': {550: 'ppm'}}
Index(['calcium', 'magnesium'], dtype='object')
0.31135592559569936
2.179297665026459
{'crop': {549: 'Basic Drinking Water Analysis (WHO)'}, 'chemical_name':

In [106]:
result

{'CA028WA0129': [{'sample_code': 'CA028WA0129',
   'status': 'warning',
   'message': 'Analysis not in models',
   'details': 'Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models'}],
 'CA028WA0130': [{'sample_code': 'CA028WA0130',
   'status': 'warning',
   'message': 'Analysis not in models',
   'details': 'Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models'}],
 'CA074WA0076': [{'sample_code': 'CA074WA0076',
   'status': 'warning',
   'message': 'Analysis not in models',
   'details': 'Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models'}],
 'CA309WA0185': [{'sample_code': 'CA309WA0185',
   'status': 'warning',
   'message': 'Analysis not in models',
   'details': 'Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models'}],
 'CA373WA0077': [{'sample_code': 'CA373WA0077',
   'status': 'warning',
   'message': 'Missing parameters for analysis_id: KEBS Drin

In [ ]:
result

In [123]:
result = {"CA028WA0129": [{"sample_code": "CA028WA0129", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CA028WA0130": [{"sample_code": "CA028WA0130", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CA074WA0076": [{"sample_code": "CA074WA0076", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CA309WA0185": [{"sample_code": "CA309WA0185", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CA373WA0077": [{"sample_code": "CA373WA0077", "status": "warning", "message": "Missing parameters for analysis_id: KEBS Drinking Water Microbiology", "details": "Expected parameters are e_coli,electrical_conductivity,strontium,total_coliforms for analysis: KEBS Drinking Water Microbiology"}], "CB016WA0028": [{"sample_code": "CB016WA0028", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CB075WA0029": [{"sample_code": "CB075WA0029", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CB387WA0001": [{"sample_code": "CB387WA0001", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CB387WA0002": [{"sample_code": "CB387WA0002", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CB387WA0003": [{"sample_code": "CB387WA0003", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CB387WA0004": [{"sample_code": "CB387WA0004", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CB387WA0005": [{"sample_code": "CB387WA0005", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CB387WA0006": [{"sample_code": "CB387WA0006", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CB387WA0007": [{"sample_code": "CB387WA0007", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CB387WA0008": [{"sample_code": "CB387WA0008", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CD048WA0110": [{"sample_code": "CD048WA0110", "status": "pass", "message": "Mahalanobis distance within threshold", "description": "Mahalanobis distance of 0.3319501004886362 is within threshold of 2.179297665026459 for analysis: Basic Drinking Water Analysis (WHO)"}, {"sample_code": "CD048WA0110", "status": "warning", "message": "Missing parameters for analysis_id: KEBS Drinking Water Microbiology", "details": "Expected parameters are e_coli,electrical_conductivity,strontium,total_coliforms for analysis: KEBS Drinking Water Microbiology"}], "CD053WA0323": [{"sample_code": "CD053WA0323", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CE021WA0023": [{"sample_code": "CE021WA0023", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CI042WA0783": [{"sample_code": "CI042WA0783", "status": "warning", "message": "Wrong units provided", "details": "Expected units are {'total_coliforms': {'crop': {631: 'Standard Drinking Water + Free Chlorine Analysis'}, 'chemical_name': {631: 'total_coliforms'}, 'unit_name': {631: 'mpn/100ml'}}} for analysis: Standard Drinking Water + Free Chlorine Analysis"}], "CM421WA0030": [{"sample_code": "CM421WA0030", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CP072WA0051": [{"sample_code": "CP072WA0051", "status": "warning", "message": "Missing parameters for analysis_id: Reverse Osmosis Water Use", "details": "Expected parameters are ammonium,arsenic for analysis: Reverse Osmosis Water Use"}], "CR003WA0008": [{"sample_code": "CR003WA0008", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CS333WA0505": [{"sample_code": "CS333WA0505", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CS333WA0514": [{"sample_code": "CS333WA0514", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CS333WA0515": [{"sample_code": "CS333WA0515", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CS333WA0516": [{"sample_code": "CS333WA0516", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CS333WA0517": [{"sample_code": "CS333WA0517", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}], "CT426WA0003": [{"sample_code": "CT426WA0003", "status": "pass", "message": "Mahalanobis distance within threshold", "description": "Mahalanobis distance of 0.31135592559569936 is within threshold of 2.179297665026459 for analysis: Basic Drinking Water Analysis (WHO)"}], "CT426WA0004": [{"sample_code": "CT426WA0004", "status": "fail", "message": "Mahalanobis distance exceeds threshold", "description": "Mahalanobis distance of 5.858731464060086 exceeds threshold of 2.179297665026459 for analysis: Basic Drinking Water Analysis (WHO)"}], "CT426WA0005": [{"sample_code": "CT426WA0005", "status": "pass", "message": "Mahalanobis distance within threshold", "description": "Mahalanobis distance of 0.20121398654537073 is within threshold of 2.179297665026459 for analysis: Basic Drinking Water Analysis (WHO)"}], "CT426WA0006": [{"sample_code": "CT426WA0006", "status": "fail", "message": "Mahalanobis distance exceeds threshold", "description": "Mahalanobis distance of 2.3647232278242956 exceeds threshold of 2.179297665026459 for analysis: Basic Drinking Water Analysis (WHO)"}], "CT426WA0007": [{"sample_code": "CT426WA0007", "status": "pass", "message": "Mahalanobis distance within threshold", "description": "Mahalanobis distance of 0.20778528929412454 is within threshold of 2.179297665026459 for analysis: Basic Drinking Water Analysis (WHO)"}], "CV002WA0269": [{"sample_code": "CV002WA0269", "status": "warning", "message": "Analysis not in models", "details": "Analysis: Standard Drinking Water Analysis (WHO) is not in the list of defined models"}]}

In [124]:
type({"dv":"dv"}) == dict

True

In [125]:
len(result.keys())

33

In [126]:
len(test)

37

In [127]:
pd.DataFrame([ i[[j for j in i.keys()][0]]  for i in result])

AttributeError: 'str' object has no attribute 'keys'

In [128]:
pd.DataFrame([ result[i][0]  for i in result.keys()])

,sample_code,status,message,details,description
0,CA028WA0129,warning,Analysis not in models,Analysis: Standard Drinking Water Analysis (WH...,NaN
1,CA028WA0130,warning,Analysis not in models,Analysis: Standard Drinking Water Analysis (WH...,NaN
2,CA074WA0076,warning,Analysis not in models,Analysis: Standard Drinking Water Analysis (WH...,NaN
3,CA309WA0185,warning,Analysis not in models,Analysis: Standard Drinking Water Analysis (WH...,NaN
4,CA373WA0077,warning,Missing parameters for analysis_id: KEBS Drink...,"Expected parameters are e_coli,electrical_cond...",NaN
5,CB016WA0028,warning,Analysis not in models,Analysis: Standard Drinking Water Analysis (WH...,NaN
6,CB075WA0029,warning,Analysis not in models,Analysis: Standard Drinking Water Analysis (WH...,NaN
7,CB387WA0001,warning,Analysis not in models,Analysis: Standard Drinking Water Analysis (WH...,NaN
8,CB387WA0002,warning,Analysis not in models,Analysis: Standard Drinking Water Analysis (WH...,NaN
9,CB387WA0003,warning,Analysis not in models,Analysis: Standard Drinking Water Analysis (WH...,NaN


In [116]:
[j for j in [l for l in result.keys()]]

['CA028WA0129',
 'CA028WA0130',
 'CA074WA0076',
 'CA309WA0185',
 'CA373WA0077',
 'CB016WA0028',
 'CB075WA0029',
 'CB387WA0001',
 'CB387WA0002',
 'CB387WA0003',
 'CB387WA0004',
 'CB387WA0005',
 'CB387WA0006',
 'CB387WA0007',
 'CB387WA0008',
 'CD048WA0110',
 'CD053WA0323',
 'CE021WA0023',
 'CI042WA0783',
 'CM421WA0030',
 'CP072WA0051',
 'CR003WA0008',
 'CS333WA0505',
 'CS333WA0514',
 'CS333WA0515',
 'CS333WA0516',
 'CS333WA0517',
 'CT426WA0003',
 'CT426WA0004',
 'CT426WA0005',
 'CT426WA0006',
 'CT426WA0007',
 'CV002WA0269']